# FAKE NEWS DETECTOR
### A fake news detector using keras and neural networks.
### I'm going to use as primary features the title and the author of the article combined in a single feature.

In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot

In [36]:
# data collected thanks to Kaggle datasets.
train_data = pd.read_csv("data/train.csv")
test_data = pd.read_csv("data/train.csv")

In [37]:
train_data.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


## Data preprocessing

In [38]:
# to do the preprocessing I create a copy of the datasets
train_copy = train_data.copy()
test_copy = test_data.copy()

# fill missing values with an empty string
train_copy = train_copy.fillna('')
test_copy = test_copy.fillna('')

In [39]:
# create a new column with title and author
train_copy['final'] = train_copy['author'] + ' ' + train_copy['title']
test_copy['final'] = test_copy['author'] + ' ' + test_copy['title']

In [40]:
# download the stopwords, which are the common words that are not useful for our goal
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\domy-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [41]:
# stemming map words to their root forms
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []

In [42]:
# applying stemming and some preprocessing
for i in range(len(train_copy)):
  review = re.sub('[^a-zA-Z]',' ',train_copy['final'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)

In [43]:
# applying stemming and some preprocessing to the test data
corpus_test = []
for i in range(len(test_copy)):
  review = re.sub('[^a-zA-Z]',' ',test_copy['final'][i])
  review = review.lower()
  review = review.split()
  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus_test.append(review)

In [44]:
# converting to one hot representation
onehot_rep = [one_hot(words,voc_size)for words in corpus]
onehot_rep_test = [one_hot(words,voc_size)for words in corpus_test]

In [45]:
# padding Sentences to make them of same size
embedded_docs = pad_sequences(onehot_rep,padding='pre',maxlen=25)
embedded_docs_test = pad_sequences(onehot_rep_test,padding='pre',maxlen=25)

In [46]:
# split our training dataset to X and y
X = train_copy.drop('label',axis=1)
y=train_copy['label']
y_test = test_copy['label']
print(X.shape)
print(y.shape)

(20800, 5)
(20800,)


# Creating and training model

In [47]:
# I have used embedding layers with LSTM
model = Sequential()
model.add(Embedding(voc_size,40,input_length=25))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 40)            200000    
_________________________________________________________________
dropout_3 (Dropout)          (None, 25, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                6464      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                

In [48]:
#Converting into numpy array
X_final = np.array(embedded_docs)
y_final = np.array(y)
test_final = np.array(embedded_docs_test)
X_final.shape,y_final.shape,test_final.shape

((20800, 25), (20800,), (20800, 25))

In [49]:
#training model
model.fit(X_final,y_final,epochs=20,batch_size=64)

Epoch 1/20
325/325 [==============================] - 15s 35ms/step - loss: 0.3431 - accuracy: 0.8316
Epoch 2/20
325/325 [==============================] - 11s 35ms/step - loss: 0.0298 - accuracy: 0.9902
Epoch 3/20
325/325 [==============================] - 12s 36ms/step - loss: 0.0121 - accuracy: 0.9958
Epoch 4/20
325/325 [==============================] - 12s 37ms/step - loss: 0.0051 - accuracy: 0.9983
Epoch 5/20
325/325 [==============================] - 11s 35ms/step - loss: 0.0086 - accuracy: 0.9977
Epoch 6/20
325/325 [==============================] - 12s 37ms/step - loss: 0.0051 - accuracy: 0.9985
Epoch 7/20
325/325 [==============================] - 12s 37ms/step - loss: 0.0024 - accuracy: 0.9995
Epoch 8/20
325/325 [==============================] - 11s 35ms/step - loss: 0.0037 - accuracy: 0.9988
Epoch 9/20
325/325 [==============================] - 12s 36ms/step - loss: 8.3801e-04 - accuracy: 0.9997
Epoch 10/20
325/325 [==============================] - 12s 36ms/step - loss: 0

In [50]:
#results = model.predict_classes(test_final)
y_pred = (model.predict(test_final) > 0.5).astype("int32")

final_sub = pd.DataFrame()
final_sub['id']=test_copy['id']
final_sub['label'] = y_pred
print(final_sub.head())

   id  label
0   0      1
1   1      0
2   2      1
3   3      1
4   4      1


In [57]:
count = 0
for i in range(len(y_pred)):
    if y_pred[i] == y_test[i]:
        count += 1

print(count / len(y_pred))

1.0
